In [7]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm

import Flexivan_Prediction_Package


In [8]:
# Get filenames from folder
DATA_Folder = './Daily prediction/DATA'

FILENAMES = [f for f in os.listdir(DATA_Folder) if os.path.isfile(os.path.join(DATA_Folder, f))]
selected = "Latest_Test_"
selected2 = '_DETAILED'
FILENAMES = [f for f in FILENAMES if selected in f and selected2 not in f]
DATES = Flexivan_Prediction_Package.extract_datetimes_from_filenames(FILENAMES)
Filenames_DF = pd.DataFrame({
    "Filenames": FILENAMES,
    "Dates": DATES
})
Filenames_DF_Sorted = Filenames_DF.sort_values(by='Dates')
FILENAMES = list(Filenames_DF_Sorted['Filenames'])
DATES = list(Filenames_DF_Sorted['Dates'])

print(f'{len(FILENAMES)} Filenames were found')
print(FILENAMES)


250 Filenames were found
['Latest_Test_2025-02-16.csv', 'Latest_Test_2025-02-23.csv', 'Latest_Test_2025-02-25.csv', 'Latest_Test_2025-03-02.csv', 'Latest_Test_2025-03-03.csv', 'Latest_Test_2025-03-04.csv', 'Latest_Test_2025-03-05.csv', 'Latest_Test_2025-03-06.csv', 'Latest_Test_2025-03-07.csv', 'Latest_Test_2025-03-08.csv', 'Latest_Test_2025-03-09.csv', 'Latest_Test_2025-03-10.csv', 'Latest_Test_2025-03-11.csv', 'Latest_Test_2025-03-12.csv', 'Latest_Test_2025-03-13.csv', 'Latest_Test_2025-03-14.csv', 'Latest_Test_2025-03-15.csv', 'Latest_Test_2025-03-16.csv', 'Latest_Test_2025-03-17.csv', 'Latest_Test_2025-03-18.csv', 'Latest_Test_2025-03-19.csv', 'Latest_Test_2025-03-20.csv', 'Latest_Test_2025-03-21.csv', 'Latest_Test_2025-03-22.csv', 'Latest_Test_2025-03-23.csv', 'Latest_Test_2025-03-24.csv', 'Latest_Test_2025-03-25.csv', 'Latest_Test_2025-03-26.csv', 'Latest_Test_2025-03-27.csv', 'Latest_Test_2025-03-28.csv', 'Latest_Test_2025-03-29.csv', 'Latest_Test_2025-03-30.csv', 'Latest_Test_2

In [ ]:
# PARAMETERS
Sorting_Field='CHS Pickup Date'
Columns_2_Drop_From_Training = ['CHS ID', 'CTR Trip Id', 'CHS Return Dt', 'CHS Return LOC', 'CHS Pickup Date', 'CTR pick Dt', 'CTR Return Dt']


In [ ]:
# File_Analysis_Results_OBJ = Flexivan_Prediction_Package.File_Analysis_Reults(f'{DATA_Folder}/{FILENAMES[0]}', Sorting_Field, Columns_2_Drop_From_Training)
# File_Analysis_Results_OBJ.DATA_ORIG.columns


In [ ]:
# Generate transition matrix for LOT - pickup and return

def accumulate_transitions(pickup_lots, return_lots, transition_df=None):
    # Build a DataFrame from the two input columns
    df = pd.DataFrame({
        "pickup": pickup_lots,
        "return": return_lots
    })
    
    # Build the new transition counts
    new_transitions = pd.crosstab(df["pickup"], df["return"])
    
    # If no existing matrix, return the new one
    if transition_df is None:
        return new_transitions
    
    # Align both matrices so they have the same index/columns
    transition_df = transition_df.reindex(
        index=transition_df.index.union(new_transitions.index),
        columns=transition_df.columns.union(new_transitions.columns),
        fill_value=0
    )
    
    new_transitions = new_transitions.reindex(
        index=transition_df.index,
        columns=transition_df.columns,
        fill_value=0
    )
    
    # Add them together
    updated_df = transition_df + new_transitions
    return updated_df


LOT_Transition_Matrix = None
lot_index = None

print('ACCUmlating transition matrix from past data files...')
for filename in tqdm(FILENAMES):
    File_Analysis_Results_OBJ = Flexivan_Prediction_Package.File_Analysis_Reults(f'{DATA_Folder}/{filename}', Sorting_Field, Columns_2_Drop_From_Training)
    LOT_Transition_Matrix = accumulate_transitions(File_Analysis_Results_OBJ.DATA['CHS Pickup Loc'], File_Analysis_Results_OBJ.DATA['CHS Return Loc'], LOT_Transition_Matrix)

print('\nSaving LOT_Transition_Matrix to CSV...', end='')
LOT_Transition_Matrix.to_csv('LOT_Transition_Matrix.csv')
print('DONE.')